In [ ]:
%matplotlib inline
import sklearn
import numpy as np
import matplotlib.pyplot as plt

import librosa
import librosa.display

import IPython.display as ipd

import warnings
warnings.filterwarnings('ignore')

import import_ipynb
import preprocessor
import display
import classifier
import csv_file_io
import datetime

PROJECT_HOME = './'
# DRONE = 'Inspire2' # 'Bebop2', 'Spark', 'MavicPro', 'Phantom4', 'Matrice100', 'MavicAir', 'AutelEVO', 'Inspire2', 'JME', 'ParrotAnafi']
# DATASET_DIR = PROJECT_HOME + 'dataset/validation/Inspire2/Inspire2_validation_File3.wav'
RESULT_FILE_DIR = PROJECT_HOME + 'results/'
# DATASET_FILE_DIR = PROJECT_HOME + 'results/dataset/'
# TRAINING_DATASET_DIR = PROJECT_HOME + 'dataset/1. training/'
# VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/2. validation/'
# TEST_DATASET_DIR = PROJECT_HOME + 'dataset/3. test/'
# TRAINING_VALIDATION_DATASET_DIR = PROJECT_HOME + 'dataset/4. training_validation/'
# TRAINING_VALIDATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/5. training_validation_test/'
SEGMENTATION_DATASET_DIR = PROJECT_HOME + 'dataset/10. segmentation (w.o)/'
# SEGMENTATION_TEST_DATASET_DIR = PROJECT_HOME + 'dataset/7. segmentation(test)/'

# now = datetime.datetime.now()
# now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

N_MFCC = 40
KERNEL = "rbf" # "rbf", "poly", "sigmoid", "LinearSVC"

RESULT_FILE_NAME = "[gridsearchCV (" + KERNEL + ")-" + str(N_MFCC) + "] result_"

SAMPLE_RATE = 44100
FIG_SIZE = (15,5)

FRAME_LENGTH = 0.250 # 250 ms
FRAME_STRIDE = 0.250 # same with frame_length for 0% overlap

DIGITS = 3

b_visualizing = False

def save_grid_classification_performance(report_dict: dict, params: dict, date_time: datetime, standardscaler: bool):        

    if standardscaler == True:
        f = csv_file_io.open_csv(RESULT_FILE_DIR + RESULT_FILE_NAME + date_time + '.csv', 'a')
        csv_file_io.write_csv_data(f, [KERNEL + "'s gridsearchCV report (scale O)"])
        f.close()

        report_dict.to_csv(RESULT_FILE_DIR + RESULT_FILE_NAME + date_time + '.csv', mode='a')

        f = csv_file_io.open_csv(RESULT_FILE_DIR + RESULT_FILE_NAME + date_time + '.csv', 'a')    

        if KERNEL == "linear":
            csv_file_io.write_csv_data(f, ['params', params['model__C'], params['model__max_iter']])
        elif KERNEL == "rbf":
            csv_file_io.write_csv_data(f, ['params', params['model__C'], params['model__gamma'], params['model__max_iter']])
        elif KERNEL == "poly":
            csv_file_io.write_csv_data(f, ['params', params['model__C'], params['model__degree'], params['model__gamma'], params['model__max_iter']])
        elif KERNEL == "sigmoid":
            csv_file_io.write_csv_data(f, ['params', params['model__C'], params['model__gamma'], params['model__max_iter']])
        elif KERNEL == "LinearSVC":
            csv_file_io.write_csv_data(f, ['params', params['model__C'], params['model__max_iter']])
    
        csv_file_io.write_csv_data(f, [])
        f.close()   

    else:        
        f = csv_file_io.open_csv(RESULT_FILE_DIR + RESULT_FILE_NAME + date_time + '.csv', 'a')
        csv_file_io.write_csv_data(f, [KERNEL + "'s gridsearchCV report (scale X)"])
        f.close()

        report_dict.to_csv(RESULT_FILE_DIR + RESULT_FILE_NAME + date_time + '.csv', mode='a')

        f = csv_file_io.open_csv(RESULT_FILE_DIR + RESULT_FILE_NAME + date_time + '.csv', 'a')    

        if KERNEL == "linear":
            csv_file_io.write_csv_data(f, ['params', params['C'], params['max_iter']])
        elif KERNEL == "rbf":
            csv_file_io.write_csv_data(f, ['params', params['C'], params['gamma'], params['max_iter']])
        elif KERNEL == "poly":
            csv_file_io.write_csv_data(f, ['params', params['C'], params['degree'], params['gamma'], params['max_iter']])
        elif KERNEL == "sigmoid":
            csv_file_io.write_csv_data(f, ['params', params['C'], params['gamma'], params['max_iter']])
        elif KERNEL == "LinearSVC":
            csv_file_io.write_csv_data(f, ['params', params['C'], params['max_iter']])
    
        csv_file_io.write_csv_data(f, [])
        f.close()     

In [ ]:
dict_integrated_train_validation_test_signal = preprocessor.get_integrated_signal(SEGMENTATION_DATASET_DIR, type='SEGMENTATION')

# Visualizing Audio (waveform and power spectrum)
if b_visualizing == True:
    for key, val in dict_integrated_train_validation_test_signal.items():
        display.display_waveshow(val, key + "'s Training+Validation+Test Signal's Waveform")
        display.display_power_spectrum(val, key + "'s Training+Validation+Test Signal's Power Spectrum")

In [ ]:
# Feature Extraction (MFCC)
n_fft = int(SAMPLE_RATE * FRAME_LENGTH) # length of the FFT window, frame 하나당 sample의 수 / n_fft=2048, n_fft = int(SAMPLE_RATE * FRAME_LENGTH)
hop_length = int(SAMPLE_RATE * FRAME_STRIDE) # number of samples between successive frames / hop_length=512, hop_length = int(SAMPLE_RATE * FRAME_STRIDE)
n_mels = 128
f_max = 8000

dict_melspectrogram_train_validation_test_signal = {}
for key, val in dict_integrated_train_validation_test_signal.items():    
    dict_melspectrogram_train_validation_test_signal[key] = librosa.feature.melspectrogram(val, sr=SAMPLE_RATE, n_mels=n_mels, fmax=f_max, n_fft=n_fft, hop_length=hop_length)    

dict_mfccs_train_validation_test_signal = {}
for key, val in dict_melspectrogram_train_validation_test_signal.items():
    dict_mfccs_train_validation_test_signal[key] = librosa.feature.mfcc(S=librosa.power_to_db(val), sr=SAMPLE_RATE, n_mfcc=N_MFCC)

# print(dict_melspectrogram_train_validation_test_signal['AutelEVO'].shape)
# print(dict_mfccs_train_validation_test_signal['AutelEVO'].shape)

if b_visualizing == True:
    for key, val in dict_mfccs_train_validation_test_signal.items():
        display.display_mfccs(val, key +  "'s Training+Validation+Test Signal's MFCCs")  

In [ ]:
arr_train_validation_test_data = None
arr_train_validation_test_target = None

for key, val in dict_mfccs_train_validation_test_signal.items():    
    arr_mfccs_data = val.transpose()    
    arr_target = np.full((arr_mfccs_data.shape[0], 1), key)   

    if arr_train_validation_test_data is None:
        arr_train_validation_test_data = arr_mfccs_data
        arr_train_validation_test_target = arr_target        
    else:       
        arr_train_validation_test_data = np.vstack((arr_train_validation_test_data, arr_mfccs_data))
        arr_train_validation_test_target = np.vstack((arr_train_validation_test_target, arr_target))

print("Training+Validation+Test data / target : ", arr_train_validation_test_data.shape, arr_train_validation_test_target.shape)

In [ ]:
now = datetime.datetime.now()
now_date_time = now.strftime('%Y%m%d_%H%M%S')[2:]

CV = 10

list_max_iter = [10000]
list_C = [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0, 50,0, 100.0, 500.0, 1000.0]  
list_gamma = ['scale', 'auto']

b_standardscaler = True

# list_C = np.arange(5.0, 31.0, 1.0)
# list_gamma = np.arange(0.005, 0.016, 0.001)
PARAMETERS = {'model__max_iter': list_max_iter, 'model__C': list_C, 'model__gamma': list_gamma}
report, params = classifier.gridsearchCV_svm(KERNEL, CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_grid_classification_performance(report, params, now_date_time, b_standardscaler)

b_standardscaler = False

PARAMETERS = {'max_iter': list_max_iter, 'C': list_C, 'gamma': list_gamma}
report, params = classifier.gridsearchCV_svm(KERNEL, CV, PARAMETERS, arr_train_validation_test_data, arr_train_validation_test_target, b_standardscaler)
save_grid_classification_performance(report, params, now_date_time, b_standardscaler)